# Scrape issue comments from ghtorrent data base
Because it is faster ( and still takes a lot of time) we first scrape the relevant issue comment ids from the mysql data base.
Subsequently we use these ids to access the actual text data from the mongodb database.

In [1]:
import pickle
from pymongo import MongoClient
import MySQLdb
from collections import defaultdict

In [2]:
# connect to ghtorrent mysql data base
db=MySQLdb.connect(
                 host="127.0.0.1",
                 user="root",
                 passwd="enterenter"
               )
cursor = db.cursor()
cursor.execute("use ghtorrent_restore")

#connect to mongodb database
client = MongoClient(unicode_decode_error_handler='ignore')
mongodb = client.github


def get_comments_for_project(_id, date1, date2):
    sqlstr="""
            select ico.comment_id from issue_comments ico, issues iss
            where iss.repo_id = %s
            and ico.created_at BETWEEN date('%s') AND date('%s')
            and ico.issue_id=iss.id
    """%(_id, date1, date2)
    cursor.execute(sqlstr)
    return cursor.fetchall()

In [ ]:
final_partition= pickle.load(open("data/final_partition", "rb"))
repoidlist = list(final_partition.keys())

In [30]:
commentdata=defaultdict(list)

d1 = "2016-10-01"
d2 = "2017-03-01"

for repo_id in repoidlist:
    res=get_comments_for_project(repo_id, d1, d2)
    idlist=[int(x[0]) for x in res]
    for c_id in idlist:
        try:
            comment=mongodb['issue_comments'].find_one({"id": c_id})
            c_obj={}
            c_obj["comment"]=comment
            commentdata[repo_id].append(c_obj)
        except:
            pass

In [ ]:
pickle.dump(commentdata, open("issue_comments.p", "wb"))